In [24]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import json

In [ ]:

# json med rois
with open("rois_02.json", "r") as f:
    rois_02 = json.load(f)[:13]

with open("rois_03.json", "r") as f:
    rois_03 = json.load(f)[:13]


# Define chessboard patterns
patterns = [
    {"pattern_size": (11, 7), "square_size": 0.010},   # C
    {"pattern_size": (7, 5),  "square_size": 0.010},   # A
    {"pattern_size": (5, 7),  "square_size": 0.010},   # B
    {"pattern_size": (5, 15), "square_size": 0.010},   # D
]

# Termination criteria for cornerSubPix
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

path_02 = "/Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_02/data/*.png"
path_03 = "/Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_03/data/*.png"

images_02 = sorted(glob.glob(path_02))
images_03 = sorted(glob.glob(path_03))

print("02 images:", len(images_02))
print("03 images:", len(images_03))

objpoints = []               # 3D real world positions 
imgpoints_image_02 = []      # 2D detected points in images 
imgpoints_image_03 = []      # 2D detected points in images 

# looper over alle 19 par med 1 billede fra 02, og 1 billede fra 03. Tilføjer alle par til calibration, selv hvis 
#hvis ikke alle chessboards er fundet, because thats life 
for f02, f03 in zip(images_02, images_03):
    print(f"\nProcessing pair: {f02} + {f03}")

    gray02 = cv2.cvtColor(cv2.imread(f02), cv2.COLOR_BGR2GRAY)
    gray03 = cv2.cvtColor(cv2.imread(f03), cv2.COLOR_BGR2GRAY)

    corners_pair_02 = []
    corners_pair_03 = []
    objp_pair = []

    for idx, (r02, r03) in enumerate(zip(rois_02, rois_03)):
        x1, y1, x2, y2 = r02
        x1b, y1b, x2b, y2b = r03

        # roi koordinater på billederne 
        h02, w02 = gray02.shape
        h03, w03 = gray03.shape
        x1, x2 = max(0, min(x1, w02-1)), max(0, min(x2, w02-1))
        y1, y2 = max(0, min(y1, h02-1)), max(0, min(y2, h02-1))
        x1b, x2b = max(0, min(x1b, w03-1)), max(0, min(x2b, w03-1))
        y1b, y2b = max(0, min(y1b, h03-1)), max(0, min(y2b, h03-1))

        # Skippper invalid roi
        if x2 <= x1 or y2 <= y1 or x2b <= x1b or y2b <= y1b:
            print(f"  ⚠ Invalid ROI for board {idx+1}, skipping")
            continue

        crop02 = gray02[y1:y2, x1:x2]
        crop03 = gray03[y1b:y2b, x1b:x2b]

        board_found = False
        for pat in patterns:
            ps = pat["pattern_size"]
            sq = pat["square_size"]

            ok02, c02 = cv2.findChessboardCornersSB(crop02, ps) #finder chessboard corners indeni hver roi for hvert billede 
            ok03, c03 = cv2.findChessboardCornersSB(crop03, ps)

            if ok02 and ok03:
                print(f"  Board {idx+1}: pattern {ps} detected")

                c02 = cv2.cornerSubPix(crop02, c02, (11,11), (-1,-1), criteria)
                c03 = cv2.cornerSubPix(crop03, c03, (11,11), (-1,-1), criteria)

                c02_full = c02 + np.array([[x1, y1]])
                c03_full = c03 + np.array([[x1b, y1b]])

                corners_pair_02.append(c02_full)
                corners_pair_03.append(c03_full)

                objp = np.zeros((ps[0]*ps[1], 3), np.float32)
                objp[:, :2] = np.mgrid[0:ps[0], 0:ps[1]].T.reshape(-1, 2)
                objp *= sq

                objp_pair.append(objp)
                board_found = True
                break

        if not board_found:
            print(f"  ⚠ Board {idx+1} NOT FOUND — skipping")

    # gemmer til calibration selvom det er flawed like hell 
    objpoints.extend(objp_pair)
    imgpoints_image_02.extend(corners_pair_02)
    imgpoints_image_03.extend(corners_pair_03)
    print(f"  ℹ Pair added for calibration ({len(corners_pair_02)} boards detected)")


02 images: 19
03 images: 19

Processing pair: /Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_02/data/0000000000.png + /Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_03/data/0000000000.png
  Board 1: pattern (11, 7) detected
  Board 2: pattern (11, 7) detected
  Board 3: pattern (7, 5) detected
  Board 4: pattern (7, 5) detected
  Board 5: pattern (7, 5) detected
  Board 6: pattern (7, 5) detected
  Board 7: pattern (7, 5) detected
  Board 8: pattern (7, 5) detected
  Board 9: pattern (7, 5) detected
  Board 10: pattern (7, 5) detected
  Board 11: pattern (7, 5) detected
  ⚠ Board 12 NOT FOUND — skipping
  Board 13: pattern (7, 5) detected
  ℹ Pair added for calibration (12 boards detected)

Processing pair: /Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_02/data/0000000001.png + /Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_03/data/0000000001.png
  Board 1: pattern (11, 7) detected
  Board 2: patter

In [ ]:
#før vi kan calibrerer kameraerne skal vi have merged alle vores boards indtil et set af punkter pr billede, fordi tidligere
# blev nødt til at breake ethvert billede op i bounding bokses til hvert skakbræt. Så for at vi kan kalibrere kameraer
# ordentligt, bliver vi nødt til at "samle" dataen igen for hvert billede

objpoints = []
imgpoints_02 = []
imgpoints_03 = []

for f02, f03 in zip(images_02, images_03):
    gray02 = cv2.imread(f02, cv2.IMREAD_GRAYSCALE)
    gray03 = cv2.imread(f03, cv2.IMREAD_GRAYSCALE)

    corners_02 = []
    corners_03 = []
    objpoints_img = []

    for r02, r03 in zip(rois_02, rois_03):
        x1, y1, x2, y2 = r02
        x1b, y1b, x2b, y2b = r03

        crop02 = gray02[y1:y2, x1:x2]
        crop03 = gray03[y1b:y2b, x1b:x2b]

        for pat in patterns:
            ok02, c02 = cv2.findChessboardCornersSB(crop02, pat["pattern_size"])
            ok03, c03 = cv2.findChessboardCornersSB(crop03, pat["pattern_size"])
            if ok02 and ok03:
                c02 = cv2.cornerSubPix(crop02, c02, (11,11), (-1,-1), criteria) + np.array([[x1, y1]], dtype=np.float32)
                c03 = cv2.cornerSubPix(crop03, c03, (11,11), (-1,-1), criteria) + np.array([[x1b, y1b]], dtype=np.float32)
                corners_02.append(c02)
                corners_03.append(c03)

                objp = np.zeros((pat["pattern_size"][0]*pat["pattern_size"][1],3), np.float32)
                objp[:,:2] = np.mgrid[0:pat["pattern_size"][0],0:pat["pattern_size"][1]].T.reshape(-1,2) * pat["square_size"]
                objpoints_img.append(objp)
                break

    if corners_02:
        imgpoints_02.append(np.vstack(corners_02))
        imgpoints_03.append(np.vstack(corners_03))
        objpoints.append(np.vstack(objpoints_img))


In [ ]:
#kalibrering af begge kameraer 

ret02, mtx02, dist02, rvecs02, tvecs02 = cv2.calibrateCamera(objpoints, imgpoints_02, gray02.shape[::-1], None, None)
ret03, mtx03, dist03, rvecs03, tvecs03 = cv2.calibrateCamera(objpoints, imgpoints_03, gray03.shape[::-1], None, None)


In [4]:
#stereo calibration, finder camera extrinsics (rotation R og translation T)

image_size = (gray02.shape[1], gray02.shape[0])  # (width, height)
print("\nImage size:", image_size)

criteria_stereo = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS,
                   100,
                   1e-5)


ret, mtx02, dist02, mtx03, dist03, R, T, E, F = cv2.stereoCalibrate(
    objpoints,
    imgpoints_image_02,
    imgpoints_image_03,
    mtx02, dist02,
    mtx03, dist03,
    image_size,
    criteria=criteria_stereo,
    flags=cv2.CALIB_FIX_INTRINSIC  # har allerede intrinsics
)

print("\nStereo calibration resultater:")
print("Rotation:\n", R)
print("Translation:\n", T)


Image size: (1392, 512)

Stereo calibration resultater:
Rotation:
 [[ 0.89906385 -0.37258989 -0.22991513]
 [ 0.33765504  0.92436191 -0.17760666]
 [ 0.27869924  0.08204773  0.95686723]]
Translation:
 [[-1.35645537]
 [ 3.21200521]
 [-1.17976744]]


In [5]:
#stereo rectification 

R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    cameraMatrix1=mtx02,
    distCoeffs1=dist02,
    cameraMatrix2=mtx03,
    distCoeffs2=dist03,
    imageSize=image_size,
    R=R,
    T=T,
    flags=cv2.CALIB_ZERO_DISPARITY,
    alpha=0  # 0 = no black borders, 1 = keep all image content
)

print("stereo Rectification resultater:")
print("Q matrix (for 3D reconstruction):\n", Q)

# Build undistort/rectify maps for fast remapping
map02_x, map02_y = cv2.initUndistortRectifyMap(
    mtx02, dist02, R1, P1, image_size, cv2.CV_32FC1
)

map03_x, map03_y = cv2.initUndistortRectifyMap(
    mtx03, dist03, R2, P2, image_size, cv2.CV_32FC1
)



stereo Rectification resultater:
Q matrix (for 3D reconstruction):
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  5.52992077e+01]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -1.29413735e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.12181089e+04]
 [ 0.00000000e+00  0.00000000e+00 -2.71674944e-01  0.00000000e+00]]


In [ ]:
#formattering af resultaterne fra rectification så vi kan sammenligne det 

def print_matrix(name, M):
    """Pretty print a matrix in one line, scientific notation."""
    M_flat = M.reshape(-1)
    print(f"{name}: " + " ".join([f"{v:.6e}" for v in M_flat]))

def print_vector(name, v):
    v_flat = v.reshape(-1)
    print(f"{name}: " + " ".join([f"{x:.6e}" for x in v_flat]))


# kamera 02
print("S_02:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("K_02", mtx02)
print_vector("D_02", dist02)
print_matrix("R_02", R)                # global stereo rotation applied to cam02
print_vector("T_02", T)

# rectified 02 
print("S_rect_02:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("R_rect_02", R1)
print_matrix("P_rect_02", P1)

print("\n")

# kamera 03
print("S_03:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("K_03", mtx03)
print_vector("D_03", dist03)
print_matrix("R_03", np.eye(3))        # cam03 reference rotation (identity)
print_vector("T_03", np.zeros((3,1)))  # reference camera at origin

# rectified 03
print("S_rect_03:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("R_rect_03", R2)
print_matrix("P_rect_03", P2)

# TODO tjek op på hvordan jeg får flere skakbrætter med 




=================== YOUR CALIBRATION (MATCHING PROVIDED FORMAT) ===================

S_02: 1.392000e+03 5.120000e+02
K_02: 1.998960e+03 0.000000e+00 7.526702e+02 0.000000e+00 1.428453e+03 2.314281e+02 0.000000e+00 0.000000e+00 1.000000e+00
D_02: 9.660750e+01 -3.116005e+03 5.408466e-01 -2.655764e+00 -8.686708e+03
R_02: 8.990638e-01 -3.725899e-01 -2.299151e-01 3.376550e-01 9.243619e-01 -1.776067e-01 2.786992e-01 8.204773e-02 9.568672e-01
T_02: -1.356455e+00 3.212005e+00 -1.179767e+00
S_rect_02: 1.392000e+03 5.120000e+02
R_rect_02: 9.811850e-01 5.924398e-02 -1.837557e-01 -1.260003e-01 9.176255e-01 -3.769448e-01 1.462872e-01 3.930059e-01 9.078251e-01
P_rect_02: 1.121811e+04 0.000000e+00 -5.529921e+01 0.000000e+00 0.000000e+00 1.121811e+04 1.294137e+03 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00


S_03: 1.392000e+03 5.120000e+02
K_03: 3.923678e+02 0.000000e+00 6.043924e+02 0.000000e+00 8.557116e+02 3.753683e+02 0.000000e+00 0.000000e+00 1.000000e+00
D_03: -5.576430e+0